# SgRNA Activity Scoring

After the database had been created and all sgRNAs identified, I scored the sgRNAs using published sgRNA scoring methods. 

## Doench Score

The first method I used was the on-target activity scoring method developed by Doench et al. in <a href="http://www.nature.com/nbt/journal/v32/n12/full/nbt.3026.html">this paper</a>. The calc_doench_score function was downloaded as a python script from the Broad Institute's <a href="http://www.broadinstitute.org/rnai/public/analysis-tools/sgrna-design-v1">sgRNA Designer</a>. 

In [ ]:
import math

### Doench Score ###

"""
Calculates the on-target score for an sgRNA
Input: 30mer
Output: On-target score
Run as: python on_target_score_calculator.py <30mer>
"""
def calc_doench_score(s):
    s_list = list(s)
    s_20mer = s[4:24]
    nuc_hash = {'A':0, 'T':1, 'C':2, 'G':3}
    score = 0.597636154
    gc = s_20mer.count('G')+s_20mer.count('C')
    gc_low = -0.202625894
    gc_high = -0.166587752
    if gc < 10:
        gc_val = abs(gc-10)
        score = score+(gc_val*gc_low)
    elif gc > 10:
        gc_val = gc-10
        score = score+(gc_val*gc_high)
    #rows[1-30]cols['ATCG']
    sing_nuc_hash = {'G2':-0.275377128,'A3':-0.323887456,'C3':0.172128871,'C4':-0.100666209,'C5':-0.20180294, \
                    'G5':0.245956633,'A6':0.036440041,'C6':0.098376835,'C7':-0.741181291,\
                    'G7':-0.393264397,'A12':-0.466099015,'A15':0.085376945,'C15':-0.013813972,\
                    'A16':0.272620512,'C16':-0.119022648,'T16':-0.285944222,'A17':0.097454592,\
                    'G17':-0.17554617,'C18':-0.345795451,'G18':-0.678096426,'A19':0.22508903,\
                    'C19':-0.507794051,'G20':-0.417373597,'T20':-0.054306959,'G21':0.379899366,\
                    'T21':-0.090712644,'C22':0.057823319,'T22':-0.530567296,'T23':-0.877007428,\
                    'C24':-0.876235846,'G24':0.278916259,'T24':-0.403102218,'A25':-0.077300704,\
                    'C25':0.287935617,'T25':-0.221637217,'G28':-0.689016682,'T28':0.117877577,\
                    'C29':-0.160445304,'G30':0.386342585}
    #score_mat = np.matrix('0 0 0 0;0 0 0 -0.275377128;-0.323887456 0 0.172128871 0;0 0 -0.100666209 0;
    #0 0 -0.20180294 0.245956633;0.036440041 0 0.098376835 0;0 0 -0.741181291 -0.393264397;0 0 0 0;0 0 0 0;
    #0 0 0 0;0 0 0 0;-0.466099015 0 0 0;0 0 0 0;0 0 0 0;0.085376945 0 -0.013813972 0;
    #0.272620512 -0.285944222 -0.119022648 0;0.097454592 0 0 -0.17554617;0 0 -0.345795451 -0.678096426;
    #0.22508903 0 -0.507794051 0;0 -0.054306959 0 -0.417373597;0 -0.090712644 0 0.379899366;
    #0 -0.530567296 0.057823319 0;0 -0.877007428 0 0;0 -0.403102218 -0.876235846 0.278916259;
    #-0.077300704 -0.221637217 0.287935617 0;0 0 0 0;0 0 0 0;0 0.117877577 0 -0.689016682;
    #0 0 -0.160445304 0;0 0 0 0.386342585')
    dinuc_hash = {'GT2':-0.625778696,'GC5':0.300043317,'AA6':-0.834836245,'TA6':0.760627772,
                  'GG7':-0.490816749,'GG12':-1.516907439,'TA12':0.7092612,'TC12':0.496298609,
                  'TT12':-0.586873894,'GG13':-0.334563735,'GA14':0.76384993,'GC14':-0.53702517,
                  'TG17':-0.798146133,'GG19':-0.66680873,'TC19':0.353183252,'CC20':0.748072092,
                  'TG20':-0.367266772,'AC21':0.568209132,'CG21':0.329072074,'GA21':-0.836456755,
                  'GG21':-0.782207584,'TC22':-1.029692957,'CG23':0.856197823,'CT23':-0.463207679,
                  'AA24':-0.579492389,'AG24':0.649075537,'AG25':-0.077300704,'CG25':0.287935617,
                  'TG25':-0.221637217,'GT27':0.117877577,'GG29':-0.697740024}
    for i,nuc in enumerate(s_list):
        key = nuc+str(i+1)
        if sing_nuc_hash.has_key(key):
            nuc_score = sing_nuc_hash[key]
        else:
            nuc_score = 0
        #nuc_score = score_mat[i,nuc_hash[nuc]]
        score = score+nuc_score
        if i<29:
            dinuc = nuc+s[i+1]+str(i+1)
            if dinuc in dinuc_hash.keys():
                score = score+dinuc_hash[dinuc]
    partial_score = math.e**-score
    final_score = 1/(1+partial_score)
    return final_score

The scoring function was then run on every long sgRNA.

In [ ]:
import data_processing as dp

def run_doench(db_name, sql_version="MySQL", firewall=False):
    db_con = dp.DatabaseConnection(sql_version, db_name=db_name, firewall=firewall)
    
    rows = db_con.fetch_query("SELECT SgID, LongSg FROM SgRNATargetInformation")
    
    doench_dict = {"DoenchScore": []}
    sg_dict = {"SgID": [], "LongSg": []}
    for row in rows:
        if sql_version == "MSSQL":
            sgID = row.SgID
            longSg = row.LongSg
        else:
            sgID, longSg = row
            longSg = str(longSg)
        d_score = calc_doench_score(longSg)
        doench_dict["DoenchScore"] += [d_score]
        sg_dict["SgID"] += [sgID]
        sg_dict["LongSg"] += [longSg]
    db_con.update_many_rows(doench_dict, sg_dict, "SgRNATargetInformation")
    db_con.close_cursor()
    db_con.close_connection()

In [ ]:
run_doench("miR-test", firewall=True)

### Add MaxDoenchScore

Once the Doench score had been calculated for each extended sgRNA, the maximum Doench score for the sgRNA was determined.

In [ ]:
import data_processing as dp

def find_max_doench(db_name, sql_version="MySQL", firewall=False):
    db_con = dp.DatabaseConnection(sql_version, db_name=db_name, firewall=firewall)
    rows = db_con.fetch_query("""SELECT s.SgID, t.DoenchScore 
FROM SingleGuideRNA AS s 
JOIN SgRNATargetInformation AS t 
ON s.SgID = t.SgID""")
    max_dict = {}
    for row in rows:
        if sql_version == "MSSQL":
            sg = row.SgID
            score = row.DoenchScore
        else:
            sg, score = row
        if sg in max_dict:
            if score > max_dict[sg]:
                max_dict[sg] = score
            else:
                pass
        else:
            max_dict[sg] = score
    sg_dict = {"SgID": []}
    do_dict = {"MaxDoenchScore": []}
    for key, val in max_dict.iteritems():
        sg_dict["SgID"] += [key]
        do_dict["MaxDoenchScore"] += [val]
        
    db_con.update_many_rows(do_dict, sg_dict, "SingleGuideRNA")
    db_con.close_cursor()
    db_con.close_connection()

In [ ]:
find_max_doench("miR-test", firewall=True)

## Azimuth Score

This on-target scoring method was updated in 2015 (<a href="https://doi.org/10.1101/021568">Preprint</a>)/2016 (<a href="https://doi.org/10.1038/nbt.3437">Nature Biotechnology Paper</a>) with the Azimuth score. I contacted Microsoft and recieved an API key which enabled me to access the API and submit requests for the scores of my sgRNAs. The documentation for this API is <a href="https://studio.azureml.net/apihelp/workspaces/ee5485c1d9814b8d8c647a89db12d4df/webservices/72e5e606de0b4fa0bcde57666f0ddcba/endpoints/c24d128abfaf4832abf1e7ef45db4b54/score">here</a>. The source code is now up at <a href="https://github.com/MicrosoftResearch/Azimuth">GitHub</a> and there is a <a href="http://www.broadinstitute.org/rnai/public/analysis-tools/sgrna-design">web interface</a> as well. 

### From flat file

It seems they tweaked the scoring over time. The Azimuth scores from the API are now slightly different than the original and the API is no longer being supported. The older score will therefore be loaded from a flat file.

In [ ]:
import pandas as pd
import data_processing as dp

def import_azimuth_score(db_name, sql_version="MSSQL", firewall=False):
    """
        Fetches Azimuth score from flat file
    """
    load_dict = {"SgID": [], "PriID": [], "SgStart": []}
    az_dict = {"AzimuthScore": []}
    
    df = pd.read_csv("sgRNA Scoring/SgRNATargetInformation_table.csv", header=0, index_col=0)
    for sgID, row in df.iteritems():
        load_dict["SgID"] += [sgID]
        load_dict["PriID"] += [row["PriID"]]
        load_dict["SgStart"] += [int(row["SgStart"])]
        az_dict["AzimuthScore"] += [float(row["AzimuthScore"])]
    
    db_con = dp.DatabaseConnection(sql_version, db_name=db_name, firewall=firewall)
    db_con.update_many_rows(az_dict, load_dict, "SgRNATargetInformation")
    db_con.close_cursor()
    db_con.close_connection()

In [ ]:
import_azimuth_score("miR-test", firewall=True)

### From Azimuth API

The score can be fetched using the API and imported into a new column.

In [ ]:
import urllib2
import math
import time
import json
import data_processing as dp

### Azimuth (updated Doench) Score ###

def get_azimuth_score(api_key_file, db_name, sql_version="MySQL", firewall=False):
        
    db_con = dp.DatabaseConnection(sql_version, db_name=db_name, firewall=firewall)
    
    # create new column
    db_con.add_column("AzimuthScorev2", "FLOAT", "SgRNATargetInformation")

    STEP = 1000
    
    rows = db_con.fetch_query("SELECT SgID, PriID, SgStart, LongSg FROM SgRNATargetInformation")
    
    sg_dict = {"SgID": [], "LongSg": [], "PriID": [], "SgStart": []}
    az_dict = {"AzimuthScorev2": []}
    for row in rows:
        if sql_version == "MSSQL":
            sg_dict["SgID"] += [row.SgID]
            sg_dict["LongSg"] += [row.LongSg]
            sg_dict["PriID"] += [row.PriID]
            sg_dict["SgStart"] += [row.SgStart]
        else:
            sgID, priID, sgStart, longSg = row
            sg_dict["SgID"] += [sgID]
            sg_dict["LongSg"] += [str(longSg)]
            sg_dict["PriID"] += [str(priID)]
            sg_dict["SgStart"] += [sgStart]
    
    num_sg = len(sg_dict["LongSg"])
    num_chunks = int(math.ceil(num_sg/float(STEP)))
    for n in range(num_chunks):
        longSg_start = n*STEP
        longSg_end = min(n*STEP+STEP, num_sg)
        longSg_chunk = sg_dict["LongSg"][longSg_start:longSg_end]
        
        sgRNAs = [[sg.upper(), -1, -1] for sg in longSg_chunk]

        data = {"Inputs": {
                            "input1":{
                                      "ColumnNames": ["sequence", "cutsite", "percentpeptide"],
                                      "Values": sgRNAs
                                     },        
                          },
                "GlobalParameters": {}
               }

        body = str.encode(json.dumps(data))

        url = 'https://ussouthcentral.services.azureml.net/workspaces/ee5485c1d9814b8d8c647a89db12d4df/services/c24d128abfaf4832abf1e7ef45db4b54/execute?api-version=2.0&details=true'
        with open(api_key_file, "r") as api:
            api_key = api.readline()
        headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

        req = urllib2.Request(url, body, headers)

        try:
            response = urllib2.urlopen(req)

            result = response.read()
            parsed_results = json.loads(result)
            listOfResults = parsed_results["Results"]["output2"]["value"]["Values"]
            for m in range(len(listOfResults)):
                score = float(listOfResults[m][0])
                az_dict["AzimuthScorev2"] += [score]
        except urllib2.HTTPError, error:
            print("The request failed with status code: " + str(error.code))

            # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
            print(error.info())

            print(json.loads(error.read()))
        time.sleep(30)
    db_con.update_many_rows(az_dict, sg_dict, "SgRNATargetInformation")
    db_con.close_cursor()
    db_con.close_connection()

In [ ]:
get_azimuth_score("Azimuth_API_key.txt", "miR-test", firewall=True)

## Max Azimuth Score

The maximum azimuth score for all target sites for each sgRNA. 

In [ ]:
import data_processing as dp

def find_max_azimuth(db_name, sql_version="MySQL", firewall=False):
    db_con = dp.DatabaseConnection(sql_version, db_name=db_name, firewall=firewall)
    
    rows = db_con.fetch_query("""SELECT s.SgID, t.AzimuthScore 
FROM SingleGuideRNA AS s 
JOIN SgRNATargetInformation AS t 
ON s.SgID = t.SgID""")
    
    max_dict = {}
    for row in rows:
        if sql_version == "MSSQL":
            sg = row.SgID
            score = row.AzimuthScore
        else:
            sg, score = row
        if sg in max_dict:
            if score > max_dict[sg]:
                max_dict[sg] = score
            else:
                pass
        else:
            max_dict[sg] = score
    sg_dict = {"SgID": []}
    do_dict = {"MaxAzimuthScore": []}
    for key, val in max_dict.iteritems():
        sg_dict["SgID"] += [key]
        do_dict["MaxAzimuthScore"] += [val]
        
    db_con.update_many_rows(do_dict, sg_dict, "SingleGuideRNA")
    db_con.close_cursor()
    db_con.close_connection()

In [ ]:
find_max_azimuth("miR-test", firewall=True)

## sgRNA Scorer

The Church lab also came up with an on-target activity scoring algorthm call <a href="https://crispr.med.harvard.edu/">sgRNA Scorer</a>. <a href="http://www.nature.com/nmeth/journal/v12/n9/full/nmeth.3473.html">This paper</a> describes how this method was developed. The input for the scoring is a fasta file with the sgRNA + PAM sequence. This information can be sliced from the longSg sequence using the below code:

In [ ]:
import data_processing as dp

def get_sgRNAs(out_file, db_name, sql_version="MySQL", firewall=False):
    """
        Fetches the sgRNA and PAM sequences from the database for sgRNA Scorer
    """
    db_con = dp.DatabaseConnection(sql_version, db_name=db_name, firewall=firewall)
    rows = db_con.fetch_query("SELECT SgID, LongSg FROM SgRNATargetInformation")
    db_con.close_cursor()
    db_con.close_connection()
    
    with open(out_file, "w") as fout:
        for row in rows:
            if sql_version == "MSSQL":
                sgID = row.SgID
                longSg = row.LongSg
            else:
                sgID, longSg = row
                longSg = str(longSg)
            header_str = ">{}_{}\n".format(sgID, longSg)
            fout.write(header_str)
            fout.write("{}\n".format(longSg[4:-3]))

In [ ]:
get_sgRNAs("sgRNA Scoring/miR_sgRNAs.fa", "miR-test", firewall=True)

The stand alone code can be downloaded from this <a href="https://crispr.med.harvard.edu/">website</a> and the <a href="http://svmlight.joachims.org/">svm-light binaries</a> are also necessary . The sgRNAs were then scored relative to the human exome sgRNAs included with the package.

In [ ]:
%cd "Downloaded Programs/sgRNA.Scorer.1.0"
!python scoreMySites.py "../../sgRNA Scoring/miR_sgRNAs.fa" Hg SP miR_sgRNAScorer

The scores for each sgRNA were then imported into the SingleGuideRNA table. For sgRNAs which had multiple possible PAM sites, the highest score is added to the table.

In [ ]:
import data_processing as dp

def import_sgRNAScorer(in_file, db_name, sql_version="MySQL", firewall=False):
    """
        Imports sgRNA Scorer score
    """
    score_dict = {}
    with open(in_file, "r") as fin:
        # skip header line
        fin.next()
        for line in fin:
            ele = line.strip("\n").split("\t")
            sgID = int(ele[0].split("_")[0])
            if sgID not in score_dict:
                score_dict[sgID] = float(ele[2])
            else:
                if score_dict[sgID] < float(ele[2]):
                    score_dict[sgID] = float(ele[2])
                else:
                    continue
    sg_dict = {"SgID": []}
    sg_scorer = {"SgRNAScorer": []}
    for key, val in score_dict.iteritems():
        sg_dict["SgID"] += [key]
        sg_scorer["SgRNAScorer"] += [val]
    db_con = dp.DatabaseConnection(sql_version, db_name=db_name, firewall=firewall)
    db_con.update_many_rows(sg_scorer, sg_dict, "SingleGuideRNA")
    db_con.close_cursor()
    db_con.close_connection()

In [ ]:
import_sgRNAScorer("Downloaded Programs/sgRNA.Scorer.1.0/miR_sgRNAScorer.FinalOutput.txt",
                   "miR-test", firewall=True)